In [1]:
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

In [2]:
model = load_model(r"C:\Users\18301253\Desktop\Project\CNNModel.h5")
lb = pickle.loads(open(r"C:\Users\18301253\Desktop\Project\videoclassificationbinarizer.pickle", "rb").read())
outputvideo=r"demo_output.mp4"
# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Queue = deque(maxlen=128)

In [3]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
capture_video = cv2.VideoCapture(r"s1.avi")
writer = None
(Width, Height) = (None, None)


# loop over frames from the video file stream

while True:
    # read the next frame from the file
    (taken, frame) = capture_video.read()
    
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not taken:
        break
        
    # if the frame dimensions are empty, grab them
    if Width is None or Height is None:
        (Width, Height) = frame.shape[:2]
 
    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean


    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Queue.append(preds)

    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]


    # draw the activity on the output frame
    text = "Activity: {}".format(label)
    cv2.putText(output, text, (15,20), cv2.FONT_HERSHEY_SIMPLEX,0.75, (255, 0,0), 5)

    
    # check if the video writer is None
    if writer is None:
    # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(outputvideo, fourcc, 60,(512, 512), True)
    # write the output frame to disk
    writer.write(output)
    # show the output image
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# release the file pointers
print("Finalising............")
writer.release()
capture_video.release()

Finalising............
